# Credit Card Fraud Detection::

Download dataset from this link:

https://www.kaggle.com/mlg-ulb/creditcardfraud

# Description about dataset::

The datasets contains transactions made by credit cards in September 2013 by european cardholders.
This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

It contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, we cannot provide the original features and more background information about the data. Features V1, V2, … V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-senstive learning. 


### Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise.

# WORKFLOW :

1.Load Data

2.Check Missing Values ( If Exist ; Fill each record with mean of its feature )

3.Standardized the Input Variables. 

4.Split into 50% Training(Samples,Labels) , 30% Test(Samples,Labels) and 20% Validation Data(Samples,Labels).

5.Model : input Layer (No. of features ), 3 hidden layers including 10,8,6 unit & Output Layer with activation function relu/tanh (check by experiment).

6.Compilation Step (Note : Its a Binary problem , select loss , metrics according to it)

7.Train the Model with Epochs (100).

8.If the model gets overfit tune your model by changing the units , No. of layers , epochs , add dropout layer or add Regularizer according to the need .

9.Prediction should be > 92%
10.Evaluation Step
11Prediction


# Task::

## Identify fraudulent credit card transactions.

In [40]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [41]:
import pandas as pd
import numpy as np
import tensorflow as tf
df = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/AI_Assignments/CreditCardData.csv')

In [42]:
# !pip install -U scikit-learn

In [43]:
df.head(5)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [44]:
from sklearn.model_selection import train_test_split

In [45]:
non_fraud=df[df["Class"]==0]
fraud = df[df["Class"]==1]
non_fraud=non_fraud.sample(3*fraud.shape[0])
data = fraud.append(non_fraud, ignore_index=True)
# data["Class"].value_counts()
x_data= data.drop(columns="Class", axis=0)
label = data["Class"]

In [46]:
train_data, test_data, train_labels, test_labels=train_test_split(x_data, label, test_size=0.3, random_state=1, stratify = label)

In [47]:
train_mean = train_data.mean(axis=0) # taking the mean of 
train_data -= train_mean
train_std = train_data.std(axis=0)
train_data /= train_std
test_data -= train_mean
test_data /= train_std

In [48]:
from keras import models,layers
from keras import regularizers

model = models.Sequential()
model.add(layers.Dense(20, activation='relu',kernel_regularizer=regularizers.l2(0.001), input_shape=(train_data.shape[1],)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(8, activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(6, activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

In [49]:
model.compile(loss= 'binary_crossentropy', optimizer= 'rmsprop', metrics=['accuracy'])

In [50]:
model.fit(train_data, train_labels, epochs=100,validation_split = 0.3)

Epoch 1/100
31/31 [==============================] - 1s 10ms/step - loss: 0.9695 - accuracy: 0.5195 - val_loss: 0.7090 - val_accuracy: 0.7899
Epoch 2/100
31/31 [==============================] - 0s 3ms/step - loss: 0.8121 - accuracy: 0.6054 - val_loss: 0.6774 - val_accuracy: 0.8478
Epoch 3/100
31/31 [==============================] - 0s 3ms/step - loss: 0.7564 - accuracy: 0.7057 - val_loss: 0.6562 - val_accuracy: 0.8551
Epoch 4/100
31/31 [==============================] - 0s 3ms/step - loss: 0.7113 - accuracy: 0.7353 - val_loss: 0.6341 - val_accuracy: 0.8647
Epoch 5/100
31/31 [==============================] - 0s 3ms/step - loss: 0.7136 - accuracy: 0.7628 - val_loss: 0.6026 - val_accuracy: 0.8768
Epoch 6/100
31/31 [==============================] - 0s 3ms/step - loss: 0.6520 - accuracy: 0.7723 - val_loss: 0.5810 - val_accuracy: 0.8744
Epoch 7/100
31/31 [==============================] - 0s 3ms/step - loss: 0.6474 - accuracy: 0.8065 - val_loss: 0.5579 - val_accuracy: 0.8792
Epoch 8/100


In [51]:
loss, accuracy = model.evaluate(test_data, test_labels)

19/19 [==============================] - 0s 1ms/step - loss: 0.2274 - accuracy: 0.9509


In [52]:
loss

0.2274484932422638

In [53]:
accuracy

0.950930655002594

In [54]:
x = model.predict(test_data).astype(dtype="u8")

In [55]:
a=x

In [56]:
b= np.array(test_labels)

In [57]:
a= a.reshape(1,a.shape[0])

In [58]:
np.size(a != b) - np.count_nonzero(a == b)

34

In [59]:
con = tf.math.confusion_matrix(labels= test_labels, predictions=x )

In [60]:
con

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[443,   0],
       [ 34, 114]], dtype=int32)>